对于通用方式，要自己提供提示词模板，模板中有些参数是必须的，比如，现在使用FUNCTION_CALL模式，那么，提示词模板中，必须包含一个参数agent_scratchpad.
他的作用是：它用于存储和传递Agent的思考过程。比如，在调用链式工具时
（如先搜索天气再推荐行程）， agent_scratchpad 保留所有历史步骤，避免上下文丢失。format方法
会将intermediate_steps转换为特定格式的字符串，并赋值给agent_scratchpad变量。如果不传递
intermediate_steps参数，会导致KeyError: 'intermediate_steps'错误。

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent,AgentExecutor,Tool
from langchain_community.tools.tavily_search import TavilySearchResults
import os
import dotenv
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["OPENAI_BASE_URL"]= os.getenv("OPENAI_BASE_URL")

search=TavilySearchResults(max_results=5)
search_tool=Tool(name="Search",func=search.run,description="useful for when you need to answer questions about current events")

prompt=ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant,请务必使用我提供的search_tool来获取信息"),
    ("human","{input}"),
    ("placeholder","{agent_scratchpad}"),
])
agent=create_tool_calling_agent(llm=ChatOpenAI(temperature=0,model="gpt-4o-mini"),tools=[search_tool],prompt=prompt)
agent_executor=AgentExecutor(agent=agent,tools=[search_tool],verbose=True)

result=agent_executor.invoke({"input":"请用中文回答：今天北京的天气怎么样？"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `Search` with `今天北京的天气`


[{'title': '中国气象局-天气预报- 北京', 'url': 'https://weather.cma.cn/web/weather/', 'content': '|  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |\n| 气温 | -4.5℃ | 1.7℃ | 3.8℃ | 2.9℃ | -0.3℃ | -2.5℃ | -4.7℃ | -5.7℃ |\n| 降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 |\n| 风速 | 3.1m/s | 2.8m/s | 3.2m/s | 3.3m/s | 1.9m/s | 2.9m/s | 2.7m/s | 2.9m/s |\n| 风向 | 西北风 | 西北风 | 西南风 | 西南风 | 西南风 | 西南风 | 西南风 | 西南风 |\n| 气压 | 1025hPa | 1025.4hPa | 1022.2hPa | 1021.7hPa | 1021.8hPa | 1020.8hPa | 1019.6hPa | 1017.8hPa | [...] |  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |\n| 气温 | -5.8℃ | -0.2℃ | 1.1℃ | 0.3℃ | -2.3℃ | -4.9℃ | -7.1℃ | -8.4℃ |\n| 降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 |\n| 风速 | 0.5m/s | 2m/s | 3.3m/s | 1m/s | 0.1m/s | 3.3m/s | 3m/s | 3.3m/s |\n| 风向